In [1]:
pip install xgboost lightgbm catboost optuna

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Note: you may need to restart the kernel to use updated packages.


In [74]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

import matplotlib.pyplot as plt; plt.style.use('ggplot')
import seaborn as sns

from scipy.stats import rankdata
from sklearn.cluster import KMeans
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, roc_auc_score, cohen_kappa_score, davies_bouldin_score, calinski_harabasz_score, silhouette_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

import optuna 

s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

file_key_1 = 'Tabular-Playground-Series/TS-S3-Ep5/train.csv'
file_key_2 = 'Tabular-Playground-Series/TS-S3-Ep5/test.csv'
file_key_3 = 'Tabular-Playground-Series/TS-S3-Ep5/sample_submission.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

bucket_object_3 = bucket.Object(file_key_3)
file_object_3 = bucket_object_3.get()
file_content_stream_3 = file_object_3.get('Body')

## Reading data files
train = pd.read_csv(file_content_stream_1)
test = pd.read_csv(file_content_stream_2)
submission = pd.read_csv(file_content_stream_3)

In [62]:
train = train[np.isin(train['quality'], [5, 6, 7])].reset_index(drop = True)
train['quality'].value_counts()

5    839
6    778
7    333
Name: quality, dtype: int64

In [23]:
train['quality'].value_counts() 
# / train.shape[0]

5    839
6    778
7    333
4     55
8     39
3     12
Name: quality, dtype: int64

In [11]:
train['quality'] = np.where(((train['quality'] < 5) | (train['quality'] > 7)), 4, train['quality'])
train['quality'].value_counts()

5    839
6    778
7    333
4    106
Name: quality, dtype: int64

In [12]:
train['quality'].value_counts() / train.shape[0]

5    0.408074
6    0.378405
7    0.161965
4    0.051556
Name: quality, dtype: float64

# Feature Engineering 

In [75]:
train['alcohol_density'] = train['alcohol'] * train['density']
train['sulphate/density'] = train['sulphates']  / train['density']
train['alcohol_sulphate'] = train['alcohol'] * train['sulphates']

test['alcohol_density'] = test['alcohol']  * test['density']
test['sulphate/density'] = test['sulphates']  / test['density']
test['alcohol_sulphate'] = test['alcohol'] * test['sulphates']

In [76]:
test_md = test.copy()

X = train[['sulphate/density', 'alcohol_density', 'alcohol', 'sulphates']]
# X = train.drop(columns = ['Id', 'quality'], axis = 1)
Y = train['quality'] - 3
# test_md = test_md.drop(columns = 'Id', axis = 1)
test_md = test_md[['sulphate/density', 'alcohol_density', 'alcohol', 'sulphates']]

# XGBoost Modeling

In [92]:
def objective(trial):
    
    ## Parameters to be evaluated
    param = dict(objective = 'multi:softmax',
                 eval_metric = 'mlogloss',
                 tree_method ='hist', 
                 random_state = 42,
                 max_depth = trial.suggest_int('max_depth', 2, 10),
                 learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log = True),
                 n_estimators = trial.suggest_int('n_estimators', 30, 10000),
                 gamma = trial.suggest_float('gamma', 0, 10),
                 min_child_weight = trial.suggest_int('min_child_weight', 1, 100),
                 colsample_bytree = trial.suggest_float('colsample_bytree', 0.2, 0.9),
                 subsample = trial.suggest_float('subsample', 0.2, 0.9)
                )

    scores = []

    skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state=42)
    
    for fold, (train_idx, valid_idx) in enumerate(skf.split(X, Y)):
        print(fold, end = ' ')
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train , y_valid = Y.iloc[train_idx] , Y.iloc[valid_idx]

        model = XGBClassifier(**param)
        model.fit(X_train, y_train)

        preds_valid = model.predict(X_valid)
        score = cohen_kappa_score(y_valid,  preds_valid, weights = "quadratic")
        scores.append(score)
    return np.mean(scores)

In [93]:
study = optuna.create_study(direction = "maximize")
study.optimize(objective, n_trials = 50, timeout = 3600)

[I 2023-02-03 00:22:54,818] A new study created in memory with name: no-name-7f2c1fc0-ff7b-4608-9cc5-5ac1bdb2b089


0 1 2 3 4 

[I 2023-02-03 00:23:17,987] Trial 0 finished with value: 0.5085029653995343 and parameters: {'max_depth': 7, 'learning_rate': 0.07893059761375568, 'n_estimators': 6239, 'gamma': 6.996574362263747, 'min_child_weight': 3, 'colsample_bytree': 0.6827053418368323, 'subsample': 0.8827994532173611}. Best is trial 0 with value: 0.5085029653995343.


0 1 2 3 4 

[I 2023-02-03 00:23:26,605] Trial 1 finished with value: 0.4794433632983127 and parameters: {'max_depth': 4, 'learning_rate': 0.0027765485982789414, 'n_estimators': 2192, 'gamma': 2.7654270668185674, 'min_child_weight': 69, 'colsample_bytree': 0.3716522890814079, 'subsample': 0.4392124782358095}. Best is trial 0 with value: 0.5085029653995343.


...

[I 2023-02-03 00:45:21,069] Trial 47 finished with value: 0.52442386967912 and parameters: {'max_depth': 5, 'learning_rate': 0.06448770549718999, 'n_estimators': 8587, 'gamma': 4.0241892549138765, 'min_child_weight': 53, 'colsample_bytree': 0.5972652787294238, 'subsample': 0.4109372286493358}. Best is trial 37 with value: 0.5274373986835524.


0 1 2 3 4 

[I 2023-02-03 00:45:52,474] Trial 48 finished with value: 0.5211516995739742 and parameters: {'max_depth': 6, 'learning_rate': 0.033468856471462576, 'n_estimators': 9142, 'gamma': 5.467375625464141, 'min_child_weight': 70, 'colsample_bytree': 0.7241407391058825, 'subsample': 0.37232947007738193}. Best is trial 37 with value: 0.5274373986835524.


0 1 2 3 4 

[I 2023-02-03 00:46:19,343] Trial 49 finished with value: 0.522761525250149 and parameters: {'max_depth': 3, 'learning_rate': 0.09770337536146823, 'n_estimators': 7627, 'gamma': 3.534787553175006, 'min_child_weight': 63, 'colsample_bytree': 0.6870582102458276, 'subsample': 0.49150059163150434}. Best is trial 37 with value: 0.5274373986835524.


In [101]:
XGB_cv_scores, XGB_imp = list(), list()
preds = list()

## Running 5 times CV
for i in range(5):
    
    skf = StratifiedKFold(n_splits = 5, random_state = 42, shuffle = True)
    
    for train_ix, test_ix in skf.split(X, Y):
        
        ## Splitting the data 
        X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
        Y_train, Y_test = Y.iloc[train_ix], Y.iloc[test_ix]
                
        ## Building RF model
        XGB_md = XGBClassifier(tree_method = 'hist',
                               colsample_bytree = 0.7, 
                               gamma = 5.5, 
                               learning_rate = 0.031, 
                               max_depth = 5, 
                               min_child_weight = 68, 
                               n_estimators = 8800, 
                               subsample = 0.41, 
                               random_state = 42).fit(X_train, Y_train)
        XGB_imp.append(XGB_md.feature_importances_)
        
        ## Predicting on X_test and test
        XGB_pred_1 = XGB_md.predict(X_test)
        XGB_pred_2 = XGB_md.predict(test_md)
        
        ## Computing roc-auc score
        XGB_cv_scores.append(cohen_kappa_score(Y_test, XGB_pred_1, weights = 'quadratic'))
        preds.append(XGB_pred_2)

XGB_cv_score = np.mean(XGB_cv_scores)    
print('The average roc-auc score over 5-folds (run 5 times) is:', XGB_cv_score)

The average roc-auc score over 5-folds (run 5 times) is: 0.5280753543267158


In [102]:
XGB_preds_test = pd.DataFrame(preds).mode(axis = 0).loc[0, ] + 3

submission['quality'] = XGB_preds_test.astype(int)
submission.head()

,Id,quality
0,2056,5
1,2057,6
2,2058,5
3,2059,6
4,2060,6


In [103]:
submission['quality'].value_counts()

5    616
6    564
7    192
Name: quality, dtype: int64

In [104]:
submission.to_csv('XGB_baseline_FE_9.csv', index = False)

# LightGBM

In [55]:
lgb_cv_scores, lgb_imp = list(), list()
preds = list()

## Running 5 times CV
for i in range(5):
    
    skf = StratifiedKFold(n_splits = 5, random_state = 42, shuffle = True)
    
    for train_ix, test_ix in skf.split(X, Y):
        
        ## Splitting the data 
        X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
        Y_train, Y_test = Y.iloc[train_ix], Y.iloc[test_ix]
                
        ## Building RF model
        lgb_md = LGBMClassifier(n_estimators = 1000,
                                max_depth = 5,
                                learning_rate = 0.01,
                                num_leaves = 10,
                                lambda_l1 = 3,
                                lambda_l2 = 3,
                                bagging_fraction = 0.7,
                                feature_fraction = 0.7).fit(X_train, Y_train)
        lgb_imp.append(lgb_md.feature_importances_)
        
        ## Predicting on X_test and test
        lgb_pred_1 = lgb_md.predict(X_test)
        lgb_pred_2 = lgb_md.predict(test_md)
        
        ## Computing roc-auc score
        lgb_cv_scores.append(cohen_kappa_score(Y_test, lgb_pred_1, weights = 'quadratic'))
        preds.append(lgb_pred_2)

lgb_cv_score = np.mean(lgb_cv_scores)    
print('The average oof roc-auc score over 5-folds (run 5 times) is:', lgb_cv_score)

[LightGBM] [Warning] lambda_l1 is set=3, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] lambda_l2 is set=3, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] lambda_l1 is set=3, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] lambda_l2 is set=3, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] lambda_l1 is set=3, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3
[LightGBM] [Warning] 

In [56]:
lgb_preds_test = pd.DataFrame(preds).mode(axis = 0).loc[0, ] + 5

submission['quality'] = lgb_preds_test.astype(int)
submission.head()

,Id,quality
0,2056,5
1,2057,6
2,2058,6
3,2059,6
4,2060,6


In [57]:
submission['quality'].value_counts()

5    603
6    602
7    167
Name: quality, dtype: int64